In [48]:
import requests
import pandas
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
from supabase.client import Client, create_client
import datetime
from tqdm import tqdm

import os
from dotenv import load_dotenv
load_dotenv()

True

In [40]:
pbar = tqdm(total=len(papersDF))
for index, row in papersDF.iterrows():
    pbar.update(1)
pbar.close()

100%|██████████| 4924/4924 [00:00<00:00, 58875.48it/s]


In [41]:
embedding = OpenAIEmbeddings()
supabase: Client = create_client(supabase_url=os.environ.get("SUPABASE_URL"), supabase_key=os.environ.get("SUPABASE_SERVICE_KEY"))

In [42]:
papersDF = pandas.read_csv('csv-microRNAAN-set (1).csv')

In [53]:
not_work_dois = []

In [55]:
pbar = tqdm(total=len(papersDF))
for index, row in papersDF.iterrows():
    doi = row['DOI']
    print(f"https://api.unpaywall.org/v2/{doi}?email=ryandu9221@gmail.com")
    req = requests.get(f"https://api.unpaywall.org/v2/{doi}?email=ryandu9221@gmail.com")
    url = req.json().get('best_oa_location').get('url_for_pdf')
    if url is None:
        not_work_dois.append(doi)
        # print("no url " + doi)
        continue
    try:
        fileloader = OnlinePDFLoader(url)
        loader = UnstructuredPDFLoader(fileloader.file_path)
        splitter = RecursiveCharacterTextSplitter(
            chunk_size = 500,
            chunk_overlap  = 0,
            length_function = len,
            is_separator_regex = False,
        )
        chunks = loader.load_and_split(text_splitter=splitter)
        for chunk in chunks:
            chunk.metadata["doi"] = doi
            chunk.metadata["add_date"] = str(datetime.datetime.now().date())
            chunk.metadata.pop("source")
        embedded = SupabaseVectorStore.from_documents(chunks, embedding, client=supabase, table_name="knowledge")
    except:
        not_work_dois.append(doi)
        # print("not work " + doi)
    pbar.update(1)
pbar.close()

  0%|          | 2/4924 [01:14<50:52:37, 37.21s/it]


https://api.unpaywall.org/v2/10.1038/s41584-020-0426-0?email=ryandu9221@gmail.com
hi
uhoh url https://www.nature.com/articles/s41584-020-0426-0.pdf


  0%|          | 1/4924 [00:15<21:12:13, 15.51s/it]

https://api.unpaywall.org/v2/10.1161/CIRCRESAHA.115.306300?email=ryandu9221@gmail.com


  0%|          | 2/4924 [00:16<9:07:43,  6.68s/it] 

...

not work 10.3390/ijms21124370
https://api.unpaywall.org/v2/10.1186/s12920-019-0570-z?email=ryandu9221@gmail.com
hi
uhoh url https://bmcmedgenomics.biomedcentral.com/track/pdf/10.1186/s12920-019-0570-z


  0%|          | 4/4924 [00:41<15:11:46, 11.12s/it]

https://api.unpaywall.org/v2/10.1007/s00109-021-02037-7?email=ryandu9221@gmail.com
hi
uhoh url https://link.springer.com/content/pdf/10.1007/s00109-021-02037-7.pdf


  0%|          | 5/4924 [00:46<11:53:44,  8.71s/it]

not work 10.1007/s00109-021-02037-7
https://api.unpaywall.org/v2/10.1186/s12933-022-01544-2?email=ryandu9221@gmail.com
hi
uhoh url https://cardiab.biomedcentral.com/counter/pdf/10.1186/s12933-022-01544-2


# Test Cycle

In [ ]:
test_doi = "10.1038/s41584-020-0426-0"

In [ ]:
req = requests.get(f"https://api.unpaywall.org/v2/{test_doi}?email=ryandu9221@gmail.com")
url = req.json().get('best_oa_location').get('url')
fileloader = OnlinePDFLoader(url)
loader = UnstructuredPDFLoader(fileloader.file_path)
splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 500,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)
chunks = loader.load_and_split(text_splitter=splitter)
for chunk in chunks:
    chunk.metadata["doi"] = test_doi
    chunk.metadata["add_date"] = str(datetime.datetime.now().date())
    chunk.metadata.pop("source")

In [ ]:
embedded = SupabaseVectorStore.from_documents(chunks, embedding, client=supabase, table_name="knowledge")

AttributeError: 'tuple' object has no attribute 'page_content'